# Calculate porous opacities for DSHARP constants

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import dsharp_opac as do

If we are running this non-interactively, parse the arguments from the commandline. To run it from the command line, you can execute:

    jupyter nbconvert --to python dsharp_porosity.ipynb

In [ ]:
import __main__ as main
interactive = not hasattr(main, '__file__')
if not interactive:
    RTHF   = argparse.RawTextHelpFormatter
    PARSER = argparse.ArgumentParser(description='calculate porous DSHARP opacities', formatter_class=RTHF)
    PARSER.add_argument('-p', '--porosity', help='select porositiy', type=float, default=0.0)
    PARSER.add_argument('-s', '--smoothed', help='smooth opacities by averageing around the particle size', action='store_true')
    PARSER.add_argument('-e', '--extrapol', help='extrapolate opacities for large grains (g, S1, S2 values become invalid)', action='store_true')
    
    PARSER.add_argument('-a0', help='minimum particle size',      type=float, default=1e-5)
    PARSER.add_argument('-a1', help='maximum particle size',      type=float, default=1e3)
    PARSER.add_argument('-na', help='number of size grid points', type=int,   default=100)
    
    PARSER.add_argument('-l0', help='minimum wavelength',          type=float, default=1e-5)
    PARSER.add_argument('-l1', help='maximum wavelength',          type=float, default=1e0)
    PARSER.add_argument('-nl', help='number of wavelength points', type=int,   default=150)
    
    ARGS  = PARSER.parse_args()
    
    porosity = ARGS.porosity
    smoothed = ARGS.smoothed
    extrapol = ARGS.extrapol
    
    a0 = ARGS.a0
    a1 = ARGS.a1
    na = ARGS.na
    
    l0 = ARGS.l0
    l1 = ARGS.l1
    nl = ARGS.nl

Select the properties in case we are running this as notebook

In [ ]:
if interactive:
    print('picking values interactively')
    porosity = 0.0
    smoothed = True
    extrapol = True
    
    a0 = 1e-5
    a1 = 1e3
    na = 100
    
    l0 = 1e-5
    l1 = 1e0
    nl = 150

Set file name to store

In [ ]:
fname = 'dsharp_p{:.2f}{}{}'.format(porosity, smoothed * '_smooth', extrapol * '_extrapol')

Get optical constants for the DSHARP mix:

In [ ]:
oc, rho_s = do.get_dsharp_mix(porosity=porosity)

Define size, mass, and wavelength grid. 

In [ ]:
a   = np.logspace(np.log10(a0), np.log10(a1), na)
lam = np.logspace(np.log10(l0), np.log10(l1), nl)

Calculate the opacities

In [ ]:
print('\n\n---------------------')
print('Calculating opacities')
print('---------------------')
print(f'Smoothing of opacities:   {smoothed}')
print(f'Extrapolate large grains: {extrapol}')
print(f'porosity:                 {porosity}')

In [ ]:
if smoothed:
    results = do.get_smooth_opacities(a, lam, rho_s, oc, extrapolate_large_grains=extrapol)
else:
    results = do.get_opacities(a, lam, rho_s, oc, extrapolate_large_grains=extrapol)

Store results

In [ ]:
np.savez_compressed(
    fname,
    a     = results['a'],
    lam   = results['lam'],
    k_abs = results['k_abs'],
    k_sca = results['k_sca'],
    g     = results['g'],
    rho_s = results['rho_s'],
    S1    = results['S1'],
    S2    = results['S2'],
    theta = results['theta'])